In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

from sklearn.ensemble import RandomForestRegressor
from category_encoders.james_stein import JamesSteinEncoder

from sklearn.metrics import r2_score

##### В качестве датасета используется: https://www.kaggle.com/competitions/mercedes-benz-greener-manufacturing

In [2]:
df = pd.read_csv('data/train.csv')

In [4]:
print(df.shape)
df.head()

(4209, 378)


,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


##### Бейзлайн 
В качестве бейзлайна будет использоваться отбор признаков и построение модели lightgbm с дефолтными параметрами.  
Метрика для оценки из соревнования R^2.

In [19]:
X_train, X_val, y_train, y_val = train_test_split(df.drop(['ID', 'y'], axis=1), df['y'], test_size=0.25, random_state=123)

In [20]:
cat_columns = X_train.select_dtypes(include=object).columns
num_columns = X_train.select_dtypes(exclude=object).columns

# По дефолту заполним пустые значения для непрерывных признаков медианой.
nan_encoding_num = {}
for feat in num_columns:
    nan_encoding_num[feat] = X_train[feat].median()
    X_train[feat].fillna(nan_encoding_num[feat], inplace=True)
    X_val[feat].fillna(nan_encoding_num[feat], inplace=True)

# Закодируем категориальные признаки преобразованием James-Stein
cat_encoder = JamesSteinEncoder(cols=cat_columns, return_df=True, handle_missing='value', handle_unknown='value')

X_train[cat_columns] = cat_encoder.fit_transform(X_train[cat_columns], y_train)
X_val[cat_columns] = cat_encoder.transform(X_val[cat_columns])

In [52]:
# Отберем признаки по значениям feature_importance для RandomForest

model = RandomForestRegressor(n_estimators=50, max_depth=10)
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, n_estimators=50)

In [53]:
# Отбираем топ 75% признаков на данном этапе

df_feat_import = pd.DataFrame({'feature_importance': model.feature_importances_, 'feature_names': X_train.columns})
threshold = np.percentile(df_feat_import['feature_importance'], 25)
features_1 = df_feat_import[df_feat_import['feature_importance'] >= threshold]['feature_names'].values

In [45]:
# Отбираем признаки по permutation importance так же на основе RandomForest.

# Сделаем разделение train выборки для того чтобы иметь валидационную выборку для отбора признаков.
X_train_model, X_val_model, y_train_model, y_val_model = train_test_split(X_train, y_train, test_size=0.2, random_state=123)

model = RandomForestRegressor(n_estimators=50, max_depth=10)
model.fit(X_train_model[features_1], y_train_model)

r = permutation_importance(model, X_val_model[features_1], y_val_model,
                            n_repeats=30,
                            random_state=123)


In [54]:
# Отбираем ещё топ 60% признаков на данном этапе

df_feat_import = pd.DataFrame({'feature_importance': r['importances_mean'], 'feature_names': X_train[features_1].columns})
threshold = np.percentile(df_feat_import['feature_importance'], 40)
features_2 = df_feat_import[df_feat_import['feature_importance'] >= threshold]['feature_names'].values

In [4]:
from lightgbm import LGBMRegressor

In [58]:
# Обучим модель lightgbm на отобранных признаках, посчитаем метрику R^2 на валидационной выборке.

lgb = LGBMRegressor()
lgb.fit(X_train[features_2], y_train)

LGBMRegressor()

In [60]:
y_val_pred = lgb.predict(X_val[features_2])
score = r2_score(y_val, y_val_pred)
print(f'R^2 for validation = {round(score, 3)}')

R^2 for validation = 0.559


#### В качестве первого AutoML будем использовать LightAutoML (LAMA)
https://github.com/sb-ai-lab/LightAutoML   
в нем уже есть встроенный отбор признаков и обработка NaN, категорий   

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [15]:
X_train, X_val, y_train, y_val = train_test_split(df.drop(['ID'], axis=1), df['y'], test_size=0.25, random_state=123)

In [16]:
task = Task(name='reg',
            metric=lambda y_true, y_pred: r2_score(y_true, y_pred))

automl = TabularAutoML(task=task, timeout=600)

oof_pred = automl.fit_predict(
    X_train,
    roles = {'target': 'y'}
)

c:\Users\ivan.ustinov\Anaconda3\envs\study\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\ivan.ustinov\Anaconda3\envs\study\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\ivan.ustinov\Anaconda3\envs\study\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argum

In [17]:
y_val_pred = automl.predict(X_val)
score = r2_score(y_val, y_val_pred.data)
print(f'R^2 for validation = {round(score, 3)}')

R^2 for validation = 0.582


#### В качестве второго примера AutoML возьмем Alex-autoML
https://github.com/Alex-Lekov/AutoML_Alex

In [ ]:
!pip install automl-alex

In [7]:
X_train, X_val, y_train, y_val = train_test_split(df.drop(['ID', 'y'], axis=1), df['y'], test_size=0.25, random_state=123)

In [11]:
from automl_alex import AutoMLRegressor

model = AutoMLRegressor()
model.fit(X_train.reset_index(drop=True), y_train.reset_index(drop=True), timeout=600)

18:53:46 | > Start Fit Base Model
18:54:37 | ##################################################
18:54:37 | > Start Fit Models 2
18:54:37 | ##################################################
18:54:37 | ##################################################
18:54:38 | > Step 1: calc parameters and pruned score: get test 10 trials


18:55:23 |  One iteration ~ 4.5 sec
18:55:23 |  Possible iters ~ 94.0
18:55:23 | ! Not enough time to find the optimal parameters. 
                     Possible iters < 100. 
                     Please, Increase the 'timeout' parameter for normal optimization.
18:55:23 | --------------------------------------------------
18:55:23 |   Pruned Threshold Score: 72.0421
18:55:23 | ##################################################
18:55:23 | > Step 2: Full opt with Threshold Score Pruner
18:55:23 | ##################################################
18:55:23 | > Start optimization with the parameters:
18:55:23 | CV_Folds = 7
18:55:23 | Score_CV_Folds = 3
18:55:23 | Feature_Selection = False
18:55:23 | Opt_lvl = 1
18:55:23 | Cold_start = 15
18:55:23 | Early_stoping = 25
18:55:23 | Metric = mean_squared_error
18:55:23 | Direction = minimize
18:55:23 | ##################################################
Optimize: : 0it [00:00, ?it/s]

Optimize: : 1it [00:02,  2.25s/it, | Model: LightGBM | OptScore: 67.5542 | Best mean_squared_error: 66.5076 ]

Optimize: : 2it [00:06,  3.41s/it, | Model: LightGBM | OptScore: 63.9112 | Best mean_squared_error: 66.5076 ]

Optimize: : 3it [00:08,  2.86s/it, | Model: LightGBM | OptScore: 69.4148 | Best mean_squared_error: 63.9112 ]

...

Optimize: : 134it [05:40,  2.68s/it, | Model: LightGBM | OptScore: 63.8341 | Best mean_squared_error: 63.7165 ]

Optimize: : 135it [05:43,  2.71s/it, | Model: LightGBM | OptScore: 64.2436 | Best mean_squared_error: 63.7165 ]

Optimize: : 136it [05:45,  2.47s/it, | Model: LightGBM | OptScore: 63.9227 | Best mean_squared_error: 63.7165 ]

Optimize: : 137it [05:47,  2.54s/it, | Model: LightGBM | OptScore: 64.0985 | Best mean_squared_error: 63.7165 ]

Optimize: : 138it [05:51,  2.55s/it, | Model: LightGBM | OptScore: 63.8578 | Best mean_squared_error: 63.7165 ]
19:01:15 | > Finish Opt!
19:01:15 | Best Score: 63.7165 mean_squared_error
19:01:15 | ##################################################
19:01:15 | > Fit Best Models
19:01:15 | ##################################################
19:01:21 | Save DataPrepare
19:01:26 | Save DataPrepare
19:01:30 | Save DataPrepare
19:01:36 | Save DataPrepare
19:01:42 | Save DataPrepare
19:01:42 | ##################################################
19:01:42 | > Finish!


In [12]:
y_val_pred = model.predict(X_val)
score = r2_score(y_val, y_val_pred)
print(f'R^2 for validation = {round(score, 3)}')

R^2 for validation = 0.587
